# DeepSeek-R1 on Google Colab with Ollama

Welcome! This notebook will show you how to:
1. Install **Ollama** on Google Colab.
2. Download the **DeepSeek-R1:7B** model.
3. Run some test prompts.
4. Create a small "chat" loop.
5. Measure the speed (tokens per second).

The instructions are written in simple English. Please follow them step by step.

> **Important**: Ollama has experimental support on Linux (and thus on Colab). Some things might not work smoothly. We'll do our best!

## Step 1: Check if we have a GPU (Optional)

Google Colab often provides a free GPU. If you have a GPU, this can make model inference faster.

You can check if a GPU is available by running the following cell. If it says something like "Tesla T4" or "Tesla P100" or "Tesla V100" etc., then you have a GPU!

If it shows only CPU, you can still run the model, but it may be slower.

In [ ]:
# This command shows if a GPU is connected.
!nvidia-smi

## Step 2: Install Ollama

We will use a shell script provided by Ollama to install their software on Linux (the operating system in Colab). 

In [ ]:
# Install Ollama
# The following command downloads and executes the install script from the official website.
# This might take a bit of time to run.

!curl -fsSL https://ollama.com/install.sh | sh

## Step 3: Pull (download) the DeepSeek-R1:7B model

We will now download the DeepSeek-R1:7B model. This may take some time depending on the file size and your internet connection.

> **Note**: If this step fails, it might be because:
1. The Colab session ran out of RAM or disk space.
2. Network issues.
3. The model is very large.

You can check your available Colab disk space by using the command `!df -h` if needed.

In [ ]:
# Download DeepSeek-R1 7B model
!ollama pull deepseek-r1:7b

## Step 4: Test a Simple Prompt

We will run a simple test prompt using Ollama. We'll send a quick message to see if the model responds.

### Explanation of the command:
`ollama run deepseek-r1:7b --prompt "Hello!"`
- **ollama run**: The command to run a model.
- **deepseek-r1:7b**: The name of the model we want.
- **--prompt "Hello!"**: The text we send to the model.

Give it a try below!

In [ ]:
# Let's try a simple prompt
!ollama run deepseek-r1:7b --prompt "Hello there! How are you today?"

## Step 5: Create a Chat Loop

Sometimes we want to chat with the model more than once and keep track of the conversation. One way is to:
1. Keep track of all user messages.
2. Keep track of all model responses.
3. Put them all into a single prompt each time.
4. Pass that combined text to the model.

Below is a simple Python loop that:
1. Asks you for input.
2. Builds a conversation string.
3. Calls `ollama run` with that conversation.
4. Prints out the model's response.
5. Repeats until you type `exit`.

> **Note**: This is a basic chat. It doesn't store conversation in a robust way, and each turn might be somewhat slow. If you want a more advanced chat, you may need to use `ollama serve` and then call the Ollama API from Python. But for now, this is good enough to see how the model responds to multiple prompts in sequence.

In [ ]:
import subprocess
import shlex

conversation = []  # List to store the conversation

print("Start chatting with DeepSeek-R1. Type 'exit' to quit.")
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Goodbye!")
        break
    
    # Add user message to conversation
    conversation.append("User:" + user_input)

    # Combine conversation into a single prompt
    # We'll put each turn on its own line.
    prompt_text = "\n".join(conversation) + "\nAssistant:"

    # Build the command for Ollama
    # We'll pass the combined conversation as the prompt.
    cmd = f"ollama run deepseek-r1:7b --prompt {shlex.quote(prompt_text)}"

    # Run the command and capture output
    process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()

    # Decode response
    response = stdout.decode('utf-8')

    # Print model response
    print("Model:", response.strip())

    # Add Assistant's response to conversation
    conversation.append("Assistant:" + response.strip())


## Step 6: Test Inference Speed (Tokens per Second)

We can do a quick test to see how fast the model generates tokens. This won't be perfect, but it can give us an idea.

**Method**: We will
1. Use the `time` magic in Colab (`%%time`) to measure execution time.
2. Prompt the model with a short message.
3. Then estimate how many tokens were produced.

Ollama's output in the terminal doesn't always show the token count. If you need to track it exactly, you might need advanced logging or the `--metadata` flag. For now, let's just measure how long it takes to get a short response.

In [ ]:
# We'll time a single run with a short prompt.
%%time
!ollama run deepseek-r1:7b --prompt "Explain the theory of gravity in 1-2 sentences, like I'm 12 years old."

Check the output above. You will see how many seconds the command took. If the model responded with around 50 tokens in 3 seconds, that's ~16 tokens/second. This is just a rough measurement.

### Tip
If you want more precise speed data, you can parse logs from Ollama or add advanced flags. Also, speed often depends on your Colab instance, GPU availability, the number of threads, and other factors.

# Done!

You have successfully installed Ollama, downloaded the DeepSeek-R1:7B model, tested prompts, chatted, and checked speed. Feel free to modify the notebook or add your own prompts.

Remember, Google Colab sessions can expire. If it expires, you'll need to rerun everything (including the download step). If you plan to do serious work, you might want to pay for more storage or more stable sessions.